<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Packages-and-Data" data-toc-modified-id="Import-Packages-and-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Packages and Data</a></span><ul class="toc-item"><li><span><a href="#Data-Preprocessing" data-toc-modified-id="Data-Preprocessing-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Data Preprocessing</a></span></li><li><span><a href="#Feature-Engineering" data-toc-modified-id="Feature-Engineering-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Feature Engineering</a></span></li></ul></li><li><span><a href="#Embeddings" data-toc-modified-id="Embeddings-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Embeddings</a></span><ul class="toc-item"><li><span><a href="#Get-Max-Length-of-Tokens-and-Integer-Encode-the-Documents" data-toc-modified-id="Get-Max-Length-of-Tokens-and-Integer-Encode-the-Documents-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Get Max Length of Tokens and Integer Encode the Documents</a></span></li><li><span><a href="#Pad-Documents-to-Max-Length" data-toc-modified-id="Pad-Documents-to-Max-Length-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Pad Documents to Max Length</a></span></li><li><span><a href="#Load-in-GloVe-Vectors" data-toc-modified-id="Load-in-GloVe-Vectors-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Load in GloVe Vectors</a></span></li><li><span><a href="#Load-in-Embeddings" data-toc-modified-id="Load-in-Embeddings-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Load in Embeddings</a></span></li></ul></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Modeling</a></span><ul class="toc-item"><li><span><a href="#Define-Models" data-toc-modified-id="Define-Models-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Define Models</a></span></li><li><span><a href="#LSTM-Model-1" data-toc-modified-id="LSTM-Model-1-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>LSTM Model 1</a></span><ul class="toc-item"><li><span><a href="#Model-Fitting-and-Evaluation" data-toc-modified-id="Model-Fitting-and-Evaluation-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Model Fitting and Evaluation</a></span></li></ul></li><li><span><a href="#LSTM-Model-2" data-toc-modified-id="LSTM-Model-2-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>LSTM Model 2</a></span><ul class="toc-item"><li><span><a href="#Model2-Fitting-and-Evaluation" data-toc-modified-id="Model2-Fitting-and-Evaluation-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Model2 Fitting and Evaluation</a></span></li></ul></li><li><span><a href="#RNN-Model" data-toc-modified-id="RNN-Model-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>RNN Model</a></span></li></ul></li><li><span><a href="#The-first-LSTM-model-has-0.90-accuracy,-the-second-LSTM-model-has-0.35-accuracy,-and-the-RNN-model-has-0.79-accuracy.-Based-on-the-performance-of-the-three-models,-we-decdided-the-first-LSTM-model-as-our-final-model.-Since-it-has-the-highest-accuracy" data-toc-modified-id="The-first-LSTM-model-has-0.90-accuracy,-the-second-LSTM-model-has-0.35-accuracy,-and-the-RNN-model-has-0.79-accuracy.-Based-on-the-performance-of-the-three-models,-we-decdided-the-first-LSTM-model-as-our-final-model.-Since-it-has-the-highest-accuracy-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>The first LSTM model has 0.90 accuracy, the second LSTM model has 0.35 accuracy, and the RNN model has 0.79 accuracy. Based on the performance of the three models, we decdided the first LSTM model as our final model. Since it has the highest accuracy</a></span></li></ul></div>

## Import Packages and Data

In [9]:
import pandas as pd
import warnings
import string
import re
import nltk
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from numpy import asarray
from numpy import zeros
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers import Flatten, Masking
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk_stopwords = set(stopwords.words('english'))
warnings.filterwarnings('ignore')

In [76]:
df = pd.read_excel('Behold+product+data+04262021.xlsx')
df.head(1)

,product_id,brand,brand_category,name,details,created_at,brand_canonical_url,description,brand_description,brand_name,product_active
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,Khadi Stripe Shirt-our signature shirt,NaN,2021-01-27 01:17:19.305 UTC,https://two-nyc.myshopify.com/products/white-k...,Our signature khadi shirt\navailable in black ...,Our signature khadi shirt\n\navailable in blac...,Khadi Stripe Shirt-our signature shirt,True


In [77]:
df = df[['product_id','brand','brand_category','name','description','product_active','details']]


**We will leave out created_at and brand_canonical_url columns in this project. Since description contains the same text as in brand_description, and name contains the same text as in brand_name, we will only use description and name in this case. Also around 85% of the details column are missing values, and the text contained in this columns are highly specific to each product, so we will also leave this column on in the model building process.**


In [78]:
df.head()

,product_id,brand,brand_category,name,description,product_active,details
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,Khadi Stripe Shirt-our signature shirt,Our signature khadi shirt\navailable in black ...,True,NaN
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Unknown,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,Mid-length dress with ruffles and adjustable s...,True,NaN
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,Unknown,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,False,NaN
3,01EY50E27A0P5V6KCW01XPDB43,Cariuma,Unknown,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,False,NaN
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Cariuma,Unknown,CATIBA PRO Skate Black Suede and Canvas Contra...,NaN,False,NaN


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61355 entries, 0 to 61354
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   product_id      61355 non-null  object
 1   brand           61355 non-null  object
 2   brand_category  60896 non-null  object
 3   name            61354 non-null  object
 4   description     51238 non-null  object
 5   product_active  61355 non-null  bool  
 6   details         9200 non-null   object
dtypes: bool(1), object(6)
memory usage: 2.9+ MB


### Data Preprocessing

**In preprocessing, we first converted all the text to lower case, and we removed punctuations, new line characters, common stopwords in English, words that contain only one letter, and more than one consecutive spaces. We also fill up the missing values with 'Unknown' tokens. Lastly, we performed lemmatization to group the words that share simialr meaning.**

In [80]:
columns = ['name', 'description','details']

# change text to lower case
for col in columns:
    df[col] = df[col].str.lower()

In [81]:
df['description'] = df['description'].str.replace("\n", " ")

In [82]:
df['details']=df['details'].str.replace("\n", " ")

In [83]:
#replace null values with UNKNOWN_TOKEN
df['brand_category'] = df['brand_category'].fillna('Unknown')
df['name'] = df['name'].fillna('Unknown')
df['description'] = df['description'].fillna('Unknown')
df['details']=df['details'].fillna('Unknown')

In [84]:
# Create functions used in preprocessing
def remove_punctuations(text):
    for punctuation in '!%"#$&\'()*+,-./:;<=>?@[\\]^_`{|}~':
        text = text.replace(punctuation, '')
    return text
def remove_stopwords(text):
    words = text.split(" ")
    temp = []
    for word in words:
        if word in nltk_stopwords:
            continue
        temp.append(word)
    cleaned = " ".join(temp)
    return cleaned
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    words = text.split(" ")
    temp = []
    for word in words:
        res = lemmatizer.lemmatize(word)
        temp.append(res)
    cleaned = " ".join(temp)
    return cleaned
def more_preprocess(text):
    result = re.sub(r'—', ' ', text)
    result = re.sub(r'•', ' ', result)
    result = re.sub(r'\s+', ' ', result)
    result = re.sub(r'\b[A-Z]?pattern[A-Z]?\b', '', result)
    return result

### Feature Engineering

For feature engineering part, we created `origin`, `material`, `attribute_value`, `category`, and `colors` features. 

`origin`: corresponding to the place of production of the item, we fetched this information from the description column.

`material`: corresponding to the product materials, for example polyester, cotton, silk, etc.. 

`attribute_value`: We merge `attribute_value` from the additional tags table with the product table to include information on the product's occasion and style. 

`category`: corresponding to the category of the products like shoes, bags, and scarves.

`color`: corresponding to the colors of each product.

In [85]:
# Get the information about the product origins
df['origin'] = df['description'].str.extract(r'made in\s(?P<origin>\w+\s\w+)')
df['origin'] = df['origin'].fillna('Unknown')

In [86]:
for i in range(len(df)):
    df['origin'][i] = re.sub(r'(the usa|los angeles|new york|nyc fits|our brooklyn|nyc also|united states|la fits|the u|nyc ways|long beach|la 100|nyc 97)','usa', df['origin'][i])

for i in range(len(df)):
    if len(df['origin'][i]) > 1:
        df['origin'][i] =  df['origin'][i].split()[0]

In [20]:
df.origin.value_counts()

Unknown     52510
usa          5248
italy        1144
peru          348
china         334
            ...  
layers          1
linen           1
ny              1
brooklyn        1
taiwan          1
Name: origin, Length: 104, dtype: int64

In [87]:
# Fetch information about the product materials
df['material'] = df['description'].str.extract(r'%\s(?P<material>\w+\s)')
df['material'] = df['material'].fillna('Unknown')

In [22]:
df.material.value_counts()

Unknown            41912
cotton              5060
viscose             1183
polyester           1152
elastane             930
                   ...  
premium                1
polyster               1
spandexmachine         1
cotton
                1
all                    1
Name: material, Length: 321, dtype: int64

In [88]:
# We performed the preprocessings after we retrieved information from the data for feature engineering
df['description'] = df['description'].apply(remove_punctuations)
df['description'] = df['description'].apply(remove_stopwords)
df['description'] = df['description'].apply(lemmatize)

In [89]:
df['name'] = df['name'].apply(remove_punctuations)
df['name'] = df['name'].apply(remove_stopwords)
df['name'] = df['name'].apply(lemmatize)

In [90]:
df['description'] = df['description'].apply(more_preprocess)

In [91]:
df['details'] = df['details'].apply(remove_punctuations)
df['details'] = df['details'].apply(remove_stopwords)
df['details'] = df['details'].apply(lemmatize)

In [92]:
# Import the additional tags data:
add = pd.read_csv('usc_additional_tags.csv')

In [28]:
add.head()

,product_id,product_color_id,attribute_name,attribute_value
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,linenblend
1,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,cottonblend
2,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,modern
3,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,businesscasual
4,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,classic


In [93]:
# Group the attribute_value rows together for the same product_id
grouping = add.groupby(['product_id'], as_index = False).agg({'attribute_value': ' '.join})

In [94]:
# Merge the grouped additional tags data with the product data
df = df.merge(grouping, on = 'product_id',how = 'left')

In [95]:
df['attribute_value'] = df['attribute_value'].fillna('Unknown')

In [32]:
df.head()

,product_id,brand,brand_category,name,description,product_active,origin,material,attribute_value
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,khadi stripe shirtour signature shirt,signature khadi shirt available black white ea...,True,Unknown,Unknown,Unknown
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Unknown,ruffle market dress loopy pink sistine tomato,midlength dress ruffle adjustable strap bias c...,True,usa,rose,Unknown
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,Unknown,ibi slip raw red knit sneaker woman,ibi slip raw red knit sneaker woman,False,Unknown,Unknown,Unknown
3,01EY50E27A0P5V6KCW01XPDB43,Cariuma,Unknown,ibi slip black knit sneaker woman,ibi slip black knit sneaker woman,False,Unknown,Unknown,Unknown
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Cariuma,Unknown,catiba pro skate black suede canvas contrast t...,Unknown,False,Unknown,Unknown,Unknown


In [96]:
# Create the category variable
df['category'] = 'Unknown'

In [97]:
for i in range(len(df)):
    if len(re.findall(r"\b(skirt|jeans|pants|dress|cargo|shorts|overalls|trousers|leggings)", df['description'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Bottom'
    elif len(re.findall(r"\b(shoe|boot|flat|heel|sneaker|sandal|loafer|espadrille|oxford|moccasin|monkstrap)", df['description'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Shoe'
    elif len(re.findall(r"\b(scarf|scarves)", df['description'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Scarf'
    elif len(re.findall(r"\b(bag|clutch|pouch|satchel|duffle|tote|messenger bag|shoulder bag)", df['description'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Handbag'
    elif len(re.findall(r"\b(one piece|one-piece|all-in-one|jumpsuit|playsuit|dress)", df['description'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'One Piece'
        

In [98]:
for i in range(len(df)):
    if len(re.findall(r"\b(skirt|jeans|pants|dress|cargo|shorts|overalls|trousers|leggings)", df['name'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Bottom'
    elif len(re.findall(r"\b(shoe|boot|flat|heel|sneaker|sandal|loafer|espadrille|oxford|moccasin|monkstrap)", df['name'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Shoe'
    elif len(re.findall(r"\b(scarf|scarves)", df['name'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Scarf'
    elif len(re.findall(r"\b(bag|clutch|pouch|satchel|duffle|tote|messenger bag|shoulder bag)", df['name'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Handbag'
    elif len(re.findall(r"\b(one piece|one-piece|all-in-one|jumpsuit|playsuit|dress)", df['name'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'One Piece'

In [99]:
for i in range(len(df)):
    if len(re.findall(r"\b(skirt|jeans|pants|dress|cargo|shorts|overalls|trousers|leggings)", df['details'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Bottom'
    elif len(re.findall(r"\b(shoe|boot|flat|heel|sneaker|sandal|loafer|espadrille|oxford|moccasin|monkstrap)", df['details'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Shoe'
    elif len(re.findall(r"\b(scarf|scarves)", df['details'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Scarf'
    elif len(re.findall(r"\b(bag|clutch|pouch|satchel|duffle|tote|messenger bag|shoulder bag)", df['details'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Handbag'
    elif len(re.findall(r"\b(one piece|one-piece|all-in-one|jumpsuit|playsuit|dress)", df['details'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'One Piece'

In [100]:
df['category'].value_counts()

Unknown      36696
Bottom       11951
Shoe          8343
Handbag       2878
One Piece      989
Scarf          498
Name: category, dtype: int64

In [101]:
# Create the colors variable
df['colors'] = ''

In [102]:
for i in range(len(df)):
    for col in set(re.findall(r"(beige|black|blue|brown|burgundy|gold|gray|navy|green|neutral|orange|pink|purple|silver|teal|white|yellow)", df['description'][i], flags=re.IGNORECASE)):
        df['colors'][i] = df['colors'][i] + ' ' + col
        print(f'{i} done')
    if df['colors'][i] == '':
        df['colors'][i] = 'Unknown'

0 done
0 done
3 done
15 done
15 done
16 done
40 done
43 done
44 done
50 done
54 done
60 done
60 done
63 done
63 done
64 done
65 done
65 done
67 done
68 done
72 done
72 done
73 done
73 done
74 done
74 done
76 done
77 done
78 done
78 done
79 done
80 done
81 done
81 done
83 done
83 done
86 done
87 done
88 done
89 done
99 done
99 done
100 done
101 done
101 done
102 done
102 done
105 done
106 done
106 done
107 done
107 done
108 done
108 done
113 done
113 done
113 done
114 done
115 done
115 done
116 done
116 done
122 done
131 done
132 done
134 done
147 done
148 done
150 done
158 done
161 done
161 done
162 done
162 done
163 done
163 done
164 done
164 done
165 done
165 done
168 done
168 done
170 done
171 done
171 done
172 done
172 done
173 done
173 done
174 done
175 done
175 done
176 done
177 done
177 done
178 done
178 done
179 done
179 done
180 done
180 done
181 done
181 done
181 done
182 done
182 done
186 done
188 done
191 done
191 done
193 done
234 done
373 done
373 done
374 done
374 done
3

2641 done
2642 done
2643 done
2643 done
2644 done
2644 done
2645 done
2645 done
2646 done
2646 done
2647 done
2647 done
2648 done
2649 done
2650 done
2650 done
2651 done
2653 done
2659 done
2659 done
2660 done
2676 done
2681 done
2682 done
2682 done
2683 done
2684 done
2684 done
2685 done
2685 done
2687 done
2687 done
2688 done
2688 done
2689 done
2689 done
2690 done
2690 done
2691 done
2691 done
2692 done
2692 done
2693 done
2693 done
2694 done
2699 done
2703 done
2703 done
2704 done
2704 done
2705 done
2706 done
2711 done
2712 done
2712 done
2713 done
2713 done
2715 done
2716 done
2718 done
2719 done
2720 done
2721 done
2722 done
2723 done
2723 done
2724 done
2724 done
2725 done
2725 done
2726 done
2727 done
2727 done
2728 done
2728 done
2729 done
2730 done
2730 done
2731 done
2731 done
2732 done
2732 done
2733 done
2734 done
2734 done
2735 done
2735 done
2736 done
2737 done
2740 done
2740 done
2756 done
2758 done
2760 done
2761 done
2761 done
2762 done
2763 done
2763 done
2764 done


5466 done
5472 done
5472 done
5475 done
5476 done
5477 done
5478 done
5479 done
5480 done
5481 done
5482 done
5484 done
5484 done
5485 done
5485 done
5486 done
5486 done
5514 done
5519 done
5533 done
5533 done
5538 done
5543 done
5546 done
5547 done
5548 done
5549 done
5560 done
5560 done
5570 done
5572 done
5575 done
5576 done
5577 done
5578 done
5578 done
5580 done
5580 done
5581 done
5582 done
5582 done
5583 done
5584 done
5592 done
5595 done
5596 done
5605 done
5605 done
5613 done
5613 done
5613 done
5614 done
5614 done
5623 done
5623 done
5624 done
5625 done
5626 done
5627 done
5627 done
5629 done
5639 done
5655 done
5657 done
5658 done
5672 done
5672 done
5673 done
5675 done
5678 done
5679 done
5680 done
5682 done
5683 done
5684 done
5684 done
5685 done
5686 done
5688 done
5690 done
5692 done
5694 done
5694 done
5694 done
5696 done
5701 done
5701 done
5702 done
5702 done
5703 done
5703 done
5703 done
5703 done
5704 done
5705 done
5705 done
5706 done
5707 done
5708 done
5708 done


8559 done
8561 done
8561 done
8572 done
8573 done
8574 done
8580 done
8586 done
8590 done
8594 done
8598 done
8599 done
8600 done
8602 done
8604 done
8606 done
8613 done
8614 done
8615 done
8616 done
8618 done
8619 done
8622 done
8626 done
8632 done
8633 done
8635 done
8636 done
8637 done
8637 done
8638 done
8639 done
8645 done
8650 done
8654 done
8660 done
8666 done
8667 done
8668 done
8670 done
8670 done
8672 done
8683 done
8683 done
8685 done
8687 done
8691 done
8692 done
8694 done
8697 done
8708 done
8709 done
8717 done
8720 done
8722 done
8731 done
8735 done
8735 done
8736 done
8737 done
8746 done
8746 done
8746 done
8747 done
8747 done
8767 done
8776 done
8782 done
8783 done
8787 done
8797 done
8806 done
8810 done
8818 done
8822 done
8823 done
8825 done
8826 done
8827 done
8828 done
8828 done
8829 done
8830 done
8830 done
8831 done
8832 done
8833 done
8834 done
8834 done
8835 done
8836 done
8837 done
8838 done
8839 done
8840 done
8841 done
8841 done
8841 done
8842 done
8843 done


10664 done
10665 done
10666 done
10667 done
10679 done
10679 done
10680 done
10680 done
10688 done
10689 done
10699 done
10701 done
10702 done
10710 done
10712 done
10729 done
10730 done
10731 done
10732 done
10736 done
10738 done
10745 done
10747 done
10754 done
10756 done
10757 done
10757 done
10760 done
10768 done
10769 done
10772 done
10776 done
10781 done
10789 done
10800 done
10800 done
10806 done
10820 done
10821 done
10824 done
10825 done
10825 done
10826 done
10826 done
10827 done
10827 done
10829 done
10830 done
10831 done
10832 done
10833 done
10834 done
10835 done
10836 done
10837 done
10837 done
10838 done
10839 done
10839 done
10841 done
10843 done
10843 done
10844 done
10844 done
10845 done
10845 done
10856 done
10857 done
10858 done
10862 done
10863 done
10864 done
10864 done
10868 done
10868 done
10872 done
10873 done
10877 done
10877 done
10878 done
10883 done
10883 done
10886 done
10886 done
10887 done
10896 done
10896 done
10897 done
10898 done
10898 done
10899 done

...

48257 done
48300 done
48369 done
48376 done
48377 done
48377 done
48387 done
48394 done
48443 done
48444 done
48463 done
48464 done
48474 done
48482 done
48483 done
48484 done
48520 done
48528 done
48547 done
48548 done
48549 done
48550 done
48551 done
48622 done
48654 done
48655 done
48665 done
48666 done
48666 done
48677 done
48681 done
48682 done
48682 done
48690 done
48690 done
48696 done
48697 done
48698 done
48700 done
48701 done
48702 done
48703 done
48709 done
48710 done
48712 done
48714 done
48715 done
48716 done
48716 done
48718 done
48724 done
48729 done
48729 done
48730 done
48730 done
48730 done
48731 done
48731 done
48732 done
48733 done
48733 done
48734 done
48734 done
48735 done
48735 done
48735 done
48736 done
48737 done
48738 done
48740 done
48741 done
48742 done
48743 done
48743 done
48743 done
48748 done
48762 done
48767 done
48768 done
48770 done
48771 done
48773 done
48775 done
48785 done
48785 done
48786 done
48786 done
48789 done
48789 done
48790 done
48790 done

50586 done
50591 done
50593 done
50594 done
50598 done
50600 done
50600 done
50601 done
50601 done
50612 done
50621 done
50621 done
50622 done
50625 done
50629 done
50629 done
50630 done
50631 done
50632 done
50634 done
50635 done
50636 done
50636 done
50637 done
50638 done
50638 done
50644 done
50644 done
50644 done
50645 done
50645 done
50645 done
50646 done
50646 done
50646 done
50647 done
50647 done
50647 done
50648 done
50649 done
50650 done
50651 done
50652 done
50653 done
50654 done
50654 done
50655 done
50655 done
50657 done
50669 done
50672 done
50673 done
50680 done
50681 done
50689 done
50693 done
50694 done
50695 done
50702 done
50703 done
50704 done
50705 done
50707 done
50708 done
50709 done
50711 done
50712 done
50719 done
50720 done
50720 done
50723 done
50724 done
50725 done
50728 done
50729 done
50729 done
50733 done
50735 done
50739 done
50740 done
50745 done
50745 done
50760 done
50782 done
50785 done
50786 done
50789 done
50821 done
50823 done
50825 done
50826 done

52862 done
52862 done
52865 done
52866 done
52867 done
52868 done
52870 done
52871 done
52904 done
52905 done
52906 done
52907 done
52908 done
52909 done
52909 done
52916 done
52916 done
52917 done
52917 done
52918 done
52919 done
52919 done
52920 done
52921 done
52922 done
52922 done
52922 done
52923 done
52923 done
52924 done
52925 done
52926 done
52926 done
52927 done
52927 done
52927 done
52929 done
52930 done
52930 done
52931 done
52932 done
52933 done
52933 done
52935 done
52936 done
52937 done
52938 done
52939 done
52940 done
52940 done
52940 done
52940 done
52941 done
52942 done
52942 done
52943 done
52943 done
52943 done
52943 done
52944 done
52945 done
52945 done
52946 done
52946 done
52949 done
52949 done
52950 done
52950 done
52950 done
52967 done
52981 done
52981 done
53020 done
53021 done
53021 done
53025 done
53025 done
53026 done
53027 done
53028 done
53029 done
53030 done
53031 done
53032 done
53032 done
53035 done
53035 done
53037 done
53040 done
53040 done
53042 done

56125 done
56129 done
56133 done
56133 done
56139 done
56145 done
56145 done
56146 done
56146 done
56149 done
56157 done
56162 done
56168 done
56168 done
56169 done
56169 done
56172 done
56172 done
56174 done
56181 done
56181 done
56188 done
56201 done
56201 done
56207 done
56207 done
56230 done
56231 done
56232 done
56243 done
56245 done
56246 done
56247 done
56253 done
56255 done
56255 done
56256 done
56257 done
56257 done
56259 done
56259 done
56260 done
56260 done
56261 done
56263 done
56266 done
56267 done
56267 done
56268 done
56270 done
56271 done
56274 done
56275 done
56278 done
56278 done
56279 done
56280 done
56280 done
56300 done
56305 done
56325 done
56325 done
56350 done
56353 done
56359 done
56362 done
56364 done
56365 done
56383 done
56384 done
56384 done
56391 done
56391 done
56391 done
56406 done
56406 done
56408 done
56409 done
56409 done
56409 done
56410 done
56411 done
56411 done
56412 done
56413 done
56413 done
56413 done
56414 done
56414 done
56414 done
56414 done

60545 done
60551 done
60574 done
60582 done
60644 done
60663 done
60664 done
60665 done
60666 done
60667 done
60668 done
60669 done
60670 done
60671 done
60673 done
60674 done
60675 done
60676 done
60676 done
60747 done
60755 done
60762 done
60782 done
60783 done
60784 done
60785 done
60797 done
60821 done
60822 done
60823 done
60833 done
60848 done
60849 done
60850 done
60851 done
60854 done
60868 done
60901 done
60903 done
60910 done
60911 done
60953 done
60955 done
60958 done
60959 done
60960 done
60964 done
60965 done
60965 done
60966 done
60967 done
60998 done
61017 done
61041 done
61042 done
61043 done
61045 done
61046 done
61048 done
61049 done
61050 done
61051 done
61052 done
61053 done
61054 done
61055 done
61056 done
61057 done
61059 done
61060 done
61062 done
61064 done
61066 done
61067 done
61069 done
61071 done
61073 done
61075 done
61076 done
61077 done
61078 done
61079 done
61080 done
61081 done
61082 done
61083 done
61085 done
61086 done
61087 done
61088 done
61089 done

In [103]:
for i in range(len(df)):
    for col in set(re.findall(r"(beige|black|blue|brown|burgundy|gold|gray|navy|green|neutral|orange|pink|purple|silver|teal|white|yellow)", df['details'][i], flags=re.IGNORECASE)):
        df['colors'][i] = df['colors'][i] + ' ' + col
        print(f'{i} done')
    if df['colors'][i] == '':
        df['colors'][i] = 'Unknown'

83 done
118 done
137 done
139 done
141 done
152 done
152 done
152 done
154 done
200 done
201 done
202 done
203 done
207 done
209 done
218 done
219 done
219 done
220 done
221 done
223 done
226 done
227 done
232 done
236 done
238 done
239 done
239 done
241 done
242 done
246 done
247 done
254 done
256 done
256 done
259 done
260 done
264 done
265 done
266 done
268 done
271 done
273 done
274 done
277 done
279 done
281 done
282 done
282 done
285 done
286 done
288 done
289 done
289 done
291 done
292 done
293 done
293 done
293 done
294 done
297 done
302 done
302 done
303 done
306 done
308 done
309 done
311 done
312 done
312 done
313 done
314 done
323 done
323 done
324 done
325 done
331 done
333 done
334 done
337 done
338 done
339 done
339 done
340 done
342 done
342 done
343 done
346 done
347 done
347 done
348 done
348 done
350 done
350 done
351 done
356 done
356 done
360 done
364 done
364 done
367 done
370 done
398 done
408 done
409 done
412 done
413 done
413 done
413 done
415 done
415 done
41

47350 done
47708 done
47710 done
47711 done
47715 done
47716 done
47719 done
47720 done
47721 done
47723 done
47726 done
47728 done
47732 done
47735 done
47737 done
47738 done
47740 done
47742 done
47743 done
47744 done
47745 done
47746 done
47748 done
47750 done
47752 done
47761 done
47762 done
47763 done
47767 done
47768 done
47770 done
47772 done
47773 done
47776 done
47783 done
47786 done
47787 done
47789 done
47792 done
47793 done
47794 done
47795 done
47801 done
47802 done
47803 done
47804 done
47807 done
47808 done
47809 done
47810 done
47811 done
47812 done
47813 done
47814 done
47815 done
47816 done
47818 done
47819 done
47820 done
47822 done
47824 done
47826 done
47828 done
47830 done
47831 done
47832 done
47833 done
47834 done
47835 done
47836 done
47837 done
47838 done
47839 done
47841 done
47842 done
47844 done
47845 done
47847 done
47849 done
47851 done
47853 done
47854 done
47855 done
47857 done
47859 done
47861 done
47862 done
47866 done
47867 done
47869 done
47870 done

In [39]:
df.head()

,product_id,brand,brand_category,name,description,product_active,origin,material,attribute_value,category,colors
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,khadi stripe shirtour signature shirt,signature khadi shirt available black white ea...,True,Unknown,Unknown,Unknown,Unknown,white black
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Unknown,ruffle market dress loopy pink sistine tomato,midlength dress ruffle adjustable strap bias c...,True,usa,rose,Unknown,Bottom,Unknown
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,Unknown,ibi slip raw red knit sneaker woman,ibi slip raw red knit sneaker woman,False,Unknown,Unknown,Unknown,Shoe,Unknown
3,01EY50E27A0P5V6KCW01XPDB43,Cariuma,Unknown,ibi slip black knit sneaker woman,ibi slip black knit sneaker woman,False,Unknown,Unknown,Unknown,Shoe,black
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Cariuma,Unknown,catiba pro skate black suede canvas contrast t...,Unknown,False,Unknown,Unknown,Unknown,Shoe,Unknown


In [104]:
df.to_csv('product_data.csv')

In [3]:
#df=pd.read_csv('product_data.csv')

## Embeddings

**In classification, we only took the top 30 more frequently appeared brands, and encoded the rest as 'Other'.** 

In [105]:
brands = pd.read_csv('behold_brands.csv')

In [106]:
top30 = df.groupby('brand').agg({'product_id':'count'}).sort_values(by = 'product_id', ascending = False).head(30).index

In [107]:
for i in range(len(df)):
    if df['brand'][i] in top30:
        continue
    else:
        df['brand'][i] = 'Other'

In [108]:
df['total'] = (df['brand_category'] + ' ' + df['name'] + ' ' + df['description'] +\
               ' ' + df['origin']+ ' ' + df['material']+ ' ' + df['category']+ ' ' + df['colors']+\
               ' ' + df['attribute_value']+ ' ' + df['details']).apply(str)
X = df['total'].values

In [109]:
df['description'][59796]

'fit model 59½ wearing size xsmall fabric content 75 polyester 25 elastanecategory lolux detail bollinger pant contructed buttery soft lolux fabric feature simple waistband flared leg highrise style perfect wear studio throw post practicestyle var00434'

### Get Max Length of Tokens and Integer Encode the Documents

In [110]:
def get_max_token_length_per_doc(docs):
    return max(list(map(lambda x: len(x.split()), docs)))

In [111]:
max_length = get_max_token_length_per_doc(X)

In [112]:
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(X)

In [113]:
def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

In [114]:
X = integer_encode_documents(X, tokenizer)

### Pad Documents to Max Length

In [115]:
X = pad_sequences(X, maxlen=max_length, padding='post')

In [116]:
df["brand"] = df["brand"].astype('category')
df["brand"] = df["brand"].cat.codes

In [117]:
encoder = LabelEncoder()
labels = to_categorical(encoder.fit_transform(df['brand']))

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size = 0.2, random_state = 1)

In [119]:
VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)

### Load in GloVe Vectors

In [120]:
# Load in GloVe Vectors

def load_glove_vectors():
    embeddings_index = {}
    with open('glove.6B.100d.txt',encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index


embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


### Load in Embeddings

In [121]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((VOCAB_SIZE, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector

## Modeling

### Define Models

In [ ]:
# Define the models

def make_classification_rnn_model(plot=False):
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(SimpleRNN(units=64, input_shape=(1, max_length)))
    model.add(Dense(16))
    model.add(Dense(labels.shape[1], activation='softmax'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize the model
    model.summary()
    
    if plot:
        plot_model(model, to_file='model.png', show_shapes=True)
    return model

def make_classification_lstm_model(plot=False):
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(LSTM(units=32, input_shape=(1, max_length)))
    model.add(Dense(16))
    model.add(Dense(labels.shape[1], activation='softmax'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize the model
    model.summary()
    
    if plot:
        plot_model(model, to_file='model.png', show_shapes=True)
    return model

def define_model():
    
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    model.add(LSTM(64, return_sequences=False, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    #model.add(Flatten())
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(labels.shape[1], activation='sigmoid'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

### LSTM Model 1

In [132]:
# Compile the model

model_m = make_classification_lstm_model()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 428, 100)          4996000   
_________________________________________________________________
masking_7 (Masking)          (None, 428, 100)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 32)                17024     
_________________________________________________________________
dense_12 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_13 (Dense)             (None, 31)                527       
Total params: 5,014,079
Trainable params: 18,079
Non-trainable params: 4,996,000
_________________________________________________________________


#### Model Fitting and Evaluation

In [133]:
# Fit the model

history = model_m.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)

Epoch 1/5
1381/1381 [==============================] - 317s 226ms/step - loss: 2.1210 - accuracy: 0.4590 - val_loss: 0.9418 - val_accuracy: 0.7661
Epoch 2/5
1381/1381 [==============================] - 304s 220ms/step - loss: 0.8320 - accuracy: 0.7846 - val_loss: 0.6411 - val_accuracy: 0.8383
Epoch 3/5
1381/1381 [==============================] - 299s 216ms/step - loss: 0.5697 - accuracy: 0.8514 - val_loss: 0.5148 - val_accuracy: 0.8670
Epoch 4/5
1381/1381 [==============================] - 299s 217ms/step - loss: 0.4612 - accuracy: 0.8798 - val_loss: 0.4357 - val_accuracy: 0.8861
Epoch 5/5
1381/1381 [==============================] - 292s 212ms/step - loss: 0.3659 - accuracy: 0.9018 - val_loss: 0.3864 - val_accuracy: 0.8963


In [134]:
loss, accuracy = model_m.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

384/384 [==============================] - 27s 70ms/step - loss: 0.3631 - accuracy: 0.9045
Accuracy: 90.449023


### LSTM Model 2

In [123]:

model = define_model()
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 428, 100)          4996000   
_________________________________________________________________
masking_4 (Masking)          (None, 428, 100)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 428, 64)           42240     
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_6 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_7 (Dense)              (None, 31)                527       
Total params: 5,072,831
Trainable params: 76,831
Non-trainable params: 4,996,000
_______________________________________

#### Model2 Fitting and Evaluation

In [124]:
history = model.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)

Epoch 1/5
1381/1381 [==============================] - 1576s 1s/step - loss: 16326771841876.2754 - accuracy: 0.1790 - val_loss: 0.1786 - val_accuracy: 0.2622
Epoch 2/5
1381/1381 [==============================] - 1661s 1s/step - loss: 0.1446 - accuracy: 0.2782 - val_loss: 0.1261 - val_accuracy: 0.3082
Epoch 3/5
1381/1381 [==============================] - 1658s 1s/step - loss: 0.1226 - accuracy: 0.3406 - val_loss: 0.1212 - val_accuracy: 0.3270
Epoch 4/5
1381/1381 [==============================] - 1581s 1s/step - loss: 0.1185 - accuracy: 0.3503 - val_loss: 0.1180 - val_accuracy: 0.3559
Epoch 5/5
1381/1381 [==============================] - 1515s 1s/step - loss: 0.1158 - accuracy: 0.3557 - val_loss: 0.1168 - val_accuracy: 0.3663


In [125]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

384/384 [==============================] - 68s 176ms/step - loss: 0.1198 - accuracy: 0.3547
Accuracy: 35.473880


### RNN Model

In [130]:
model_cl = make_classification_rnn_model()
history = model_cl.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)
loss, accuracy = model_cl.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 428, 100)          4996000   
_________________________________________________________________
masking_6 (Masking)          (None, 428, 100)          0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 64)                10560     
_________________________________________________________________
dense_10 (Dense)             (None, 16)                1040      
_________________________________________________________________
dense_11 (Dense)             (None, 31)                527       
Total params: 5,008,127
Trainable params: 12,127
Non-trainable params: 4,996,000
_________________________________________________________________
Epoch 1/5
1381/1381 [==============================] - 276s 198ms/step - loss: 2.1373 - accuracy: 0.4484 

## The first LSTM model has 0.90 accuracy, the second LSTM model has 0.35 accuracy, and the RNN model has 0.79 accuracy. Based on the performance of the three models, we decdided the first LSTM model as our final model. Since it has the highest accuracy